In [2]:
import jax
from neurallogic import hard_not, hard_and, harden, neural_logic_net, harden_layer, primitives

In [4]:
vmap = primitives.symbolic_vmap(lambda x: 2*x)

In [6]:
vmap([[1,2,3,4],[5,6,7,8]])

[[1, 2, 3, 4, 1, 2, 3, 4], [5, 6, 7, 8, 5, 6, 7, 8]]